# Rapport TP3 Essaim Particulaire

In [634]:
import numpy as np 
import copy
import numpy.random as rnd
import copy
import numpy.random as rnd
import time
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm


In [635]:
def cout_soudure(x):
    return 1.7781*0.625*x[1]*x[2]**2 + 0.6224*0.625*x[0]*x[2]*x[3] + 3.1661*(0.625*x[0])**2*x[3] + 19.84*(0.625*x[0])**2*x[2]


In [636]:
def g(i,x): #contraintes
    return { 1 : 0.00954*x[2]-0.625*x[1], \
             2 :  0.0193*x[2]- 0.625*x[0], \
             3 : -np.pi * x[2]**2 * x[3] - 4/3 * np.pi * x[2]**3 + 1296000 }[i]


La fonction pénalité permet de calculer le dépassement des contraintes par la particule s'il y a lieu. Elle est simplement la somme des valeurs positives des fonctions contraintes. Cette technique de pénalisation adaptative est recommandée pour les problèmes avec contraintes par les lectures scientifique referees à la fin de ce notebook.

In [637]:
def penalite(x):
    p = [max(g(i,x),0) for i in range(1,4)]

    return np.sum(p)



#### La fonction d'initialisation

In [638]:
def initiation(f,bounds,p):
   

    d=len(bounds) #finding number of dimensions
    pop=np.zeros(p) #creating empty position array
    pop=pop.tolist() #converting array to list
    pop_velocity=pop[:] #empty velocity array
    pop_val=pop[:] #empty value array
    for j in range(p): #iterating ovre the number of particles
        pop[j]=[rnd.uniform(bounds[i][0],bounds[i][1])\
                    for i in range(d)] #random coordinate within bounds
      

        pop_val[j]=f(pop[j]) + penalite(pop[j])#calculating function value
                                            #at each particle
        pop_velocity[j]=[rnd.uniform(-abs(bounds[i][1]-bounds[i][0])\
                    ,abs(bounds[i][1]-bounds[i][0])) for i in range(d)]
                    #creating random velocity values for each dimension

    global_best=pop[np.argmin(pop_val)]#getting the lowest particle value
    #global_best_faisable = penalite(global_best) == 0

    pop_best=copy.deepcopy(pop)#setting all particles current positions to best

    pop_best = [ {'particule' : pop_best[i], 'faisable' : penalite(pop_best[i]) == 0 } for i in range(len(pop_best))] # convertir les elements en une liste de dictionnaires qui nous permet de savoir si la particule respeste les contraintes ou pas
    

    return d,np.array(pop), np.array(pop_best), \
                 np.array(global_best), np.array(pop_velocity), \
                     np.array(pop_val)



#### Vérification des bornes

In [639]:
def withinbounds(bounds,pop):
   
    for i in range(len(bounds)):
        if pop[i]<bounds[i][0]: #if particle is less than lower bound
            pop[i]=bounds[i][0]
        elif pop[i]>bounds[i][1]: #if particle is more than higher bound
            pop[i]=bounds[i][1]
    return


In [640]:
def print_results(hist):
    
    scores = hist[:,1]
    best_score = min(scores)
    index = np.where(scores == best_score)
    #print(f"index : {index}")
    print(f"meilleurs dimensions : \n z1 = {0.625*hist[index[0][0],4]} \n z2 = {0.625*hist[index[0][0],5]} \n x3 = {hist[index[0][0],6]} \n x4 = {hist[index[0][0],7]}\n\n faisable = {'oui' if hist[index[0][0],2] == 1 else 'non'} \n")
    print(f"meilleur score : {best_score}\n")

   

In [641]:
def preparation_table(hist):
    hist[:,3] = [0.625 * a for a in hist[:,3]]
    hist[:,4] = [0.625 * a for a in hist[:,4]]

    table = pd.DataFrame(data=hist[:,1:], columns=["MeilleurScore","Faisabilité","NbrItrs", "z1","z2","x3","x4"], index=hist[:,0]) 
    
    return table





In [642]:
def print_table(hist):
      print("historique detaille :\n")
      return preparation_table(hist)



In [643]:
def print_analyse(hist):
    print("Analyse des donnees generees :\n")
    return preparation_table(hist).describe()


#### "faisable" : respecte les contraintes.

En comparant deux particules, une particule est meilleure si son score est inferieur et que les deux particules sont faisables ou que sa pénalité est inférieure et que les deux particules ne sont pas faisables. 

Dans le cas où un est faisable et l'autre non, on préfère toujours celle qui est faisable sans considérer le score.

Dans ma comparaison de la nouvelle position de la particule générée avec sa meilleure position, au lieu de condenser le code en vérifiant plusieurs conditions, j'ai séparé les conditions pour simplifier la lecture du code.

Le nombre d'itérations fixe par l'utilisateur n'est pas garanti d'être le nombre maximal d'itérations étant donné qu'on ne pas avoir un optimum global faisable à la fin de ces itérations. Dans ce cas, on continue à itérer jusqu'à ce qu'on tombe sur un optimum global faisable.



In [644]:
def particleswarm(f,exec, iter, bounds,p,c1,c2,vmax,tol):

    c3=c1+c2
    K=2/(abs(2-c3-np.sqrt((c3**2)-(4*c3)))) #creating velocity weighting factor
    
    historique = np.empty([0,8])
    for k in tqdm(range (exec)):

        d,pop, pop_best, global_best, pop_velocity, \
        pop_best_scores \
        = initiation(f,bounds,p) #initializing various arrays
        old_global_best=[0]*d
        
        #pop_best_scores = [pop_best_scores[i] + penalite(pop[i,:]) for i in range(len(pop))]
        
        #penal_init = penalite(global_best)
        
        j=0
        while (j < iter or abs(f(old_global_best)-f(global_best))>tol) or penalite(global_best) > 0 :
                
                j+=1
                               
                for i in range(p): #iterates over each particle
                    rp,rg=rnd.uniform(0,1,2) #creates two random numbers between 0-
                    pop_velocity[i,:]+=(c1*rp*(pop_best[i]['particule']-pop[i,:]))
                    pop_velocity[i,:]+=(c2*rg*(global_best[:]-pop[i,:]))
                    pop_velocity[i,:]=pop_velocity[i,:]*K
                    if pop_velocity[i].any() > vmax : #is any velocity is greater than vmax
                            pop_velocity[i,:]=vmax #set velocity to vmax
                    #all of the above is regarding updating the particle's velocity
                    #with regards to various parameters (local_best, p_best etc..)
                    pop[i,:]+=pop_velocity[i,:] #updating position
                    
                    withinbounds(bounds,pop[i]) #if particle is out of bounds

                    penal = penalite(pop[i])
                    particle_fitness= f(pop[i]) + penal # penalite = 0 si la particule respecte les contraites
                
                    
                    if  penal == 0 and pop_best[i]['faisable'] :
                            
                            if particle_fitness < pop_best_scores[i] :
                                pop_best[i]['particule']=pop[i,:] #checking if new best
                                pop_best_scores[i]=particle_fitness
                                
                    elif penal >0 and not pop_best[i]['faisable'] :
                               
                               if penal < penalite(pop_best[i]['particule']):
                                   pop_best[i]['particule']=pop[i,:] #checking if new best
                                   pop_best_scores[i]=particle_fitness
   
                    elif penal == 0 and not pop_best[i]['faisable'] : 
                            
                                pop_best[i]['particule']=pop[i,:] #checking if new best
                                pop_best_scores[i]=particle_fitness
                                


                    penal_g = penalite(global_best)
                    global_best_fitness = f(global_best) + penal_g           
                    
                    if (penal >0 and penal_g > 0  and penal < penal_g) or (penal == 0 and penal_g == 0 and particle_fitness < global_best_fitness) or (penal == 0 and penal_g >0 ): 

                                    old_global_best=global_best[:]
                                    global_best=copy.deepcopy(pop_best[i]['particule']) 
                                    #print('current function value: ',f(global_best))
 
                    
    
    #print('Optimum at: ',global_best,'\n','Function at optimum: ',f(global_best)) 

        historique = np.append(historique,[np.append([k,f(global_best),penalite(global_best)==0,j],[global_best])],axis=0)
        
        print(f'#global best monte-carlo iter#{k} is : {f(global_best)}')
    
    return historique


In [645]:
f=cout_soudure

bounds = [[1,99],[1,99],[10,200],[10,240]]    
p=60 #shouldn't really change 
vmax=(bounds[3][1]-bounds[3][0])*0.75

c1=2.8 #shouldn't really change
c2=1.3 #shouldn't really change
tol=0.001
exec =100
iter = 100

In [646]:
while True:
    iter = input("Entrer le nombre d'iterations par execution: ")
    epsilon = input("Entrer epsilon ( progres minimal ) : ")

    if int(iter) > 0 and float(epsilon) > 0:
      break
    else:
      print("Veuillez reesayer!")

hist = particleswarm(f,exec,int(iter),bounds,p,c1,c2,vmax,float(epsilon))    


100%|██████████| 100/100 [02:16<00:00,  1.37s/it]#global best monte-carlo iter#99 is : 6474.937813604386



In [647]:
print_results(hist)



meilleurs dimensions : 
 z1 = 0.7275909398757264 
 z2 = 0.625 
 x3 = 37.6990118872142 
 x4 = 240.0

 faisable = oui 

meilleur score : 6474.936145092796



In [648]:
print_table(hist)



historique detaille :



,MeilleurScore,Faisabilite,NbrItrs,z1,z2,x3,x4
0.0,6474.936537,1.0,68.125,0.727591,1.0,37.699013,240.000000
1.0,6474.936145,1.0,67.500,0.727591,1.0,37.699012,240.000000
2.0,6474.939137,1.0,62.500,0.727591,1.0,37.699020,240.000000
3.0,6474.937950,1.0,65.000,0.727591,1.0,37.699013,240.000000
4.0,6474.937398,1.0,66.875,0.727591,1.0,37.699014,240.000000
...,...,...,...,...,...,...,...
95.0,6543.932039,1.0,62.500,0.760711,1.0,39.415094,212.986582
96.0,6606.746805,1.0,62.500,0.791096,1.0,40.989430,190.881324
97.0,6474.936287,1.0,66.875,0.727591,1.0,37.699012,240.000000
98.0,7283.887425,1.0,62.500,1.197252,1.0,62.033743,24.489440


In [649]:
print_analyse(hist)


Analyse des donnees generees :



,MeilleurScore,Faisabilite,NbrItrs,z1,z2,x3,x4
count,100.000000,100.0,100.000000,100.000000,100.0,100.000000,100.000000
mean,6711.669193,1.0,42.351562,0.537352,1.0,44.536756,172.610997
std,320.226910,0.0,8.455102,0.118043,0.0,9.789662,86.244947
min,6474.936145,1.0,39.062500,0.454744,1.0,37.699012,10.000000
25%,6474.938555,1.0,39.062500,0.454744,1.0,37.699014,108.903846
50%,6509.642457,1.0,39.843750,0.463091,1.0,38.390929,228.727577
75%,6900.212380,1.0,41.796875,0.587649,1.0,48.717016,240.000000
max,7339.822383,1.0,100.781250,0.786780,1.0,65.225256,240.000000


### Conclusion : 

Cette métaheuristique donne de meilleurs résultats par rapport à celles utilisées dans le TP1, et s'approche d'ailleurs de l'optimum expérimentale trouvée pour ce problème qui est dans les environs de 5000.

### REFERENCES :
 1.https://www.cs.cinvestav.mx/~constraint/papers/eisci.pdf
 2.https://github.com/TomRSavage/ParticleSwarm/blob/master
 3.https://www.sciencedirect.com/science/article/pii/S0957417419305925
 4.http://ipedr.com/vol6/29-A10025.pdf